In [ ]:
using Pkg;
Pkg.activate("./")
Pkg.instantiate()

# Loading SOLPS output into IMAS data structure

In [ ]:
using SOLPS2IMAS: SOLPS2IMAS

In [ ]:
b2gmtry = "samples/b2fgmtry"
b2output = "samples/b2time.nc"
b2mn = "samples/b2mn.dat"

In [ ]:
ids = SOLPS2IMAS.solps2imas(b2gmtry, b2output; b2mn=b2mn)

# Visualising some properties

In [ ]:
using GGDUtils: GGDUtils
using Plots

IMAS ids store mesh information for edge profiles in grid_ggd. There are options to have multiple space representaions but typically you will have only one space describing the SOLPS mesh

In [ ]:
grid_ggd = ids.edge_profiles.grid_ggd[1] # First grid_ggd time slice. It is allowed to vary in time
space = grid_ggd.space[1] # First space in this grid_ggd

### Plotting grid and subsets

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(space) # Simply plot the grid described in space, all common arguments to plot can be given here

# You can overlay any subset by giving a second argument
# Labels 
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 6), markercolor=:chocolate1)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 7), linecolor=:red, linewidth=2)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 8), linecolor=:darkred, linewidth=2)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 9), linecolor=:limegreen, linewidth=2)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 10), linecolor=:darkgreen, linewidth=2)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 11), linecolor=:cyan, linewidth=2)
# plot!(space, GGDUtils.get_grid_subset(grid_ggd, 12), linecolor=:teal, linewidth=1)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 13), linecolor=:royalblue1, linewidth=2)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 14), linecolor=:navyblue, linewidth=2)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 15), linecolor=:fuchsia, linewidth=2, linestyle=:dash)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 16), linecolor=:purple4, linewidth=2, linestyle=:dash)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 101), markershape=:rect, markercolor=:royalblue1)
# plot!(space, GGDUtils.get_grid_subset(grid_ggd, 102), markershape=:rect, markercolor=:maroon)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 103), markershape=:diamond, markercolor=:fuchsia)
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 104), markershape=:diamond, markercolor=:purple4)

# Legend is supressed unless asked for specifically
plot!(legend=true)
# Default labels are subset.identifier.name but can be changed by providing a label argument


### Plotting 2D quantities as heatmaps

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

n_e = GGDUtils.get_prop_with_grid_subset_index(ids.edge_profiles.ggd[1].electrons.density, 5)
plot(ids.edge_profiles.grid_ggd, n_e, colorbar_title="Electrons density / m^(-3)")

You can overlap any grid on top of a quantity

In [ ]:

# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(ids.edge_profiles.grid_ggd, n_e) # Note default label in colorbar
plot!(space, GGDUtils.get_grid_subset(grid_ggd, 16), linecolor=:black, linewidth=2, linestyle=:solid, label="Separatix", legend=true)

# Adding equilibrium data

In [ ]:
import SD4SOLPS: SD4SOLPS

In [ ]:
eqdsk = "samples/g002296.00200"
SD4SOLPS.geqdsk_to_imas!(eqdsk, ids)

### Extrapole core profile

In [ ]:
SD4SOLPS.fill_in_extrapolated_core_profile!(ids, "electrons.density"; method="simple")
SD4SOLPS.fill_in_extrapolated_core_profile!(ids, "electrons.temperature"; method="simple")
# ... more profiles here as they become available in b2time

### Extend mesh outside SOLPS mesh to the device wall

In [ ]:

SD4SOLPS.cached_mesh_extension!(ids, eqdsk, b2gmtry)

# Loading a synthetic diagnostic

In [ ]:
using SynthDiag: SynthDiag

### Add interferometer chord details using a json file

In [ ]:
SynthDiag.add_interferometer!("samples/default_interferometer.json", ids)

### Plotting the interferometer geometry on top of SOLPS mesh

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(space)
plot!(ids.interferometer) # Default plot_type is :los 
plot!(legend=true)

You can provide custom length and thickness of mirror to be plotted and linewidth of the laser beams

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(space)
plot!(ids.interferometer, mirror_length=0.7, linewidth=4, mirror_thickness=0.2)
plot!(legend=true)

Or you can choose to omit the mirror

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(space)
plot!(ids.interferometer, mirror=false)
plot!(legend=true)

You can plot a single channel as well. You can override the in-built channel name for the label.

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(space)
plot!(ids.interferometer.channel[1], label="Channel 1")
plot!(legend=true)

### Plotting interferometer data vs time

 * Use plot_type=:n_e for integrated electron density data
 * Use plot_type=:n_e_average for averaged electron density data


In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(ids.interferometer, plot_type=:n_e)

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(ids.interferometer, plot_type=:n_e_average)

Again, to plot an individual channel, just provide the channel with correct plot_type

In [ ]:
# Choose backend
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(ids.interferometer.channel[1], plot_type=:n_e_average)

# Load Langmuir Probes

Same syntax as interferometer

In [ ]:
SynthDiag.add_langmuir_probes!("samples/default_langmuir_probes.json", ids)

 Data visualization recipes for langmuir probes have not been created yet and might not be created but you can still see them using built in plotting methodsan d knowledgeof IMAS data structure.

In [ ]:
gr()           # Fast and can save pdf
# plotlyjs()   # Use for interactive plot, can only save png

plot(ids.langmuir_probes.embedded[1].time, ids.langmuir_probes.embedded[1].n_e.data, label=ids.langmuir_probes.embedded[1].name)
plot!(ylabel="Electron density / m^(-3)", xlabel="Time / s", legend=true)